In [ ]:
######################################################################
# Begin Instructions for Setup of INCAS SDK/Testing                  #
######################################################################
#                                                                    #
# INCAS API client setup is accomplished in four steps               #
#                                                                    #
# Step 1: Certificates                                               #
# INCAS Authentication and Authorization are handled                 #
# through mutual PKI certificate verification                        #
#                                                                    #
# Below enter the local path to both the client public               #
# and the client key certificates                                    #
#                                                                    #
#Defining certificates                                               #
import imp
import json
from pprint import pprint
from incas.utils import *
from incas.request import INCASRequest
from tqdm import tqdm

certificate = 'certs/uiuc.crt.pem'     #
key_certificate = 'certs/uiuc.key.pem'  #
#                                                                    #
# Step 2: Enter the FQDN URL to the INCAS Server                     #
# Defining INCAS Host                                                #
host = 'https://incas.researchinnovations.com'               #
#                                                                    #
# Step 3: Enter the value of an ActorID available on the host this   #
# script is being executed against                                   #
#                                                                    #
# Declaring existing ActorID                                         #
existingActorID = 'bd7d788a-519b-47a2-91d5-3d83fc29bf89'             #
#                                                                    #
# Step 4: Set you PYTHONPATH environment variable                    #
# to the directory housing the INCAS SDK files (/incas)              #
# on Mac/Linux execute at command prompt:                            #
# export PYTHONPATH=<Path to SDK Dir>/incas                          #
#                                                                    #
######################################################################
# End Setup Instructions for INCAS SDK/Testing                       #
######################################################################


# Create Instance of INCAS Request/Connection
irequest = INCASRequest(host, certificate, key_certificate)

# Create Instance of INCAS Test Case Tracking
iTest = Testing()

## Add concern

In [ ]:
with open('rr.json', 'r') as f:
    data = json.load(f)

labelMap = {
    0: 'concern-3.1',
    1: 'concern-3.2',
    2: 'concern-3.3',
    3: 'concern-3.4',
    4: 'concern-3.5',
    5: 'concern-3.6',
    6: 'concern-3.7',
    7: 'concern-3.8',
    8: 'concern-3.9',
    9: 'concern-3.10',
    10: 'concern-3.11'
}

for actor in tqdm(list(data.keys())[10000: 20000]):
    annotations = [
        {
            'providerName': 'ta2-uiuc',
            'type': f'{labelMap[i]}-response-rate',
            'offsets': [],
            'text': f'Response Rate {labelMap[i]}',
            'confidence': data[actor]['attributeValue'][i]
        }
        for i in range(11)
    ]
    batchAJSON = {'annotations': annotations}
    baaResponse = irequest.postActorAnnotations(actor.split(':')[-1], batchAJSON)

## Add enrichment

In [ ]:
with open('enrich.json', 'r') as f:
    data = json.load(f)

for actor in tqdm(list(data.keys())[:10000]):
    annotations = []
    for enrich in data[actor]:
        if enrich['enrichmentName'] == 'morality' or enrich['enrichmentName'] == 'honor':
            annotations.append({
                'providerName': 'ta2-uiuc',
                'type': f'uiuc-psychographic-{enrich["enrichmentName"].lower()}-{enrich["version"]}',
                'offsets': [],
                'text': str(enrich['attributeValue']),
                'confidence': enrich['confidence'] if not isinstance(enrich['confidence'], list) else 0.5
            })
        else:
            annotations.append({
                'providerName': 'ta2-uiuc',
                'type': f'uiuc-demographic-{enrich["enrichmentName"].lower()}-{enrich["version"]}',
                'offsets': [],
                'text': str(enrich['attributeValue']),
                'confidence': enrich['confidence'] if not isinstance(enrich['confidence'], list) else 0.5
            })
    batchAJSON = {'annotations': annotations}
    baaResponse = irequest.postActorAnnotations(
        actor.split(':')[-1], batchAJSON)